In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pandas as pd
import numpy as np
from textblob import TextBlob

In [3]:
def Anime():
    
    def popular_anime(url):
        
    #Fucntion to scrape popular anime of current time from crunchyroll website and will store it in a dataframe
    
        anime=pd.DataFrame(np.zeros(120).reshape((40,3)),columns=['Anime_Name',"Number of Videos Aired",'Link'])
     
        driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32\chromedriver.exe')
    
        driver.get(url)
    
        soup=BeautifulSoup(driver.page_source,'lxml')

        content=soup.find('div',{'id':'main_content'})
    
        i=0
    
        for div in content.find_all('a',class_='portrait-element block-link titlefix'):
        
            name=div.find('span',class_='series-title block ellipsis')
            no_videos=div.find('span',class_='series-data block ellipsis')
        
            anime['Anime_Name'][i]=name.text.strip()
            anime['Number of Videos Aired'][i]=no_videos.text.strip()
            anime['Link'][i]=div['href']
            i=i+1
        
            
        driver.quit()
        
        return anime

    url_popular_anime='https://www.crunchyroll.com/videos/anime'
    anime_list=popular_anime(url_popular_anime)

    #To get the proper link of each webpage
    for i in range(len(anime_list)):
        temp=anime_list['Link'][i]
        anime_list['Link'][i]='https://www.crunchyroll.com'+temp
    

    driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32\chromedriver.exe')
    anime_list['Reviews']=anime_list['Anime_Name']

    
    #To scrape the reviews of each anime which will be used to determine the sentiment of them
    for j in range(len(anime_list)):
    
        url=anime_list['Link'][j]
        title=anime_list['Anime_Name']
    
        l=[]
    
        for i in range(1,11):
        
            url_reviews=url+'/reviews/helpful/page{}'.format(i)
            driver.get(url_reviews)
            soup=BeautifulSoup(driver.page_source,'lxml')

            content=soup.find('div',class_='user-reviews')
        
            for x in content.find_all('li',class_='white-wrapper container-shadow medium-margin-bottom'):
                review=x.find('div',class_='strong left')
                l.append(review.text)
                
            
        anime_list['Reviews'][j]=l
    
    anime_list.to_csv('Anime.csv')
    anime_list=pd.read_csv('Anime.csv')
    anime_list.drop('Unnamed: 0',axis=1,inplace=True)

    
    #Cleaning the text from unwanted characters
    for i in range(len(anime_list)):
        temp=anime_list['Reviews'][i]
        anime_list['Reviews'][i]=temp.replace('/','').replace(',','').replace('" \'','').replace('.','').replace('-','').replace('?','').replace('" \'','').replace('\'','').replace('[','').replace(']','').replace('"','')
    
    anime_list['Polarity Score']=anime_list['Anime_Name']
    anime_list['Sentiment']=anime_list['Anime_Name']

    
    #Determining sentiment score using TextBlob and classifying them according to their score
    for i in range(len(anime_list)):
        temp=TextBlob(anime_list['Reviews'][i])
        score=temp.sentiment.polarity
        anime_list['Polarity Score'][i]=score
        if score>0.5:
            anime_list['Sentiment'][i]="Very Positive Reviews"
        elif(score>0.3 and score<0.5):
            anime_list['Sentiment'][i]='Positive Reviews' 
        elif(score>0 and score<0.3):
            anime_list['Sentiment'][i]='Mixed Reviews'
        else:
            anime_list['Sentiment'][i]='Negative Reviews'
        
        
    anime_list.to_csv('Anime_Final.csv')
    
    return anime_list

In [12]:
def Manga():
    
    def popular_manga(url):
        
        #Function to scrape popular manga of current time
    
        driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32\chromedriver.exe')
    
        manga = pd.DataFrame(np.zeros(98).reshape((49,2)),columns=['Manga Name','Link'])
    
        driver.get(url)
    
        soup=BeautifulSoup(driver.page_source,'lxml')
    
        content2=soup.find('div',{'id':'main_content'})
    
        i=0
    
        for x in content2.find_all('a',class_='portrait-element block-link titlefix'):
            title=x.find('span',class_='series-title block ellipsis')
            manga['Manga Name'][i]=title.text
            manga['Link'][i]='https://www.crunchyroll.com'+x['href']
            i=i+1
            #print(title.text)
    
        return manga
    
    url_popular_manga='https://www.crunchyroll.com/comics/manga'
    manga_list=popular_manga(url_popular_manga)
    
    #To get the proper links for scraping
    #for i in range(len(manga_list)):
     #   temp=manga_list['Link'][i]
      #  manga_list['Link'][i]='https://www.crunchyroll.com'+temp
       
    #manga_list['Link']='https://www.crunchyroll.com'+manga_list['Link']   
    
    #To scrape the overview of each scraped manga
    driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32\chromedriver.exe')
    manga_list['Overview']=manga_list['Manga Name']
    for i in range(len(manga_list)):
        temp_url=manga_list['Link'][i]
        driver.get(temp_url)
        soup=BeautifulSoup(driver.page_source,'lxml')
        content9=soup.find('div',class_='right')
    
        if(content9==None):
            manga_list['Overview'][i]='Not Available'
        
        else:
            for y in content9.find_all('p',class_='description'):
            
                manga_list['Overview'][i]=y.text.strip()
                
    return manga_list
                

In [18]:
def recommendation(manga_list):
    
    #Used to convert text into computer readable form
    from sklearn.feature_extraction.text import TfidfVectorizer
    tfidf = TfidfVectorizer(stop_words='english')
    manga_list['Overview'] = manga_list['Overview'].fillna('')
    tfidf_matrix = tfidf.fit_transform(manga_list['Overview'])
    
    manga_list['ID']=manga_list['Manga Name']
    manga_list['ID']=manga_list.index+1000
    
    from sklearn.metrics.pairwise import cosine_similarity
    similarity_distance = cosine_similarity(tfidf_matrix,tfidf_matrix)
    
    return similarity_distance

def get_recommendation(title, cosine_sim):
    # Get the index of the movie that matches the title
    idx  = manga['ID'][manga['Manga Name']==title].index[0]

    # Get all manga with same similarity score
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the manga based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the manga indices
    manga_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar manga
    return manga.iloc[manga_indices]

In [21]:
def latest_news(url):
    
    driver = webdriver.Chrome(executable_path=r'C:\chromedriver_win32\chromedriver.exe')
    
    driver.get(url)
    
    soup=BeautifulSoup(driver.page_source,'lxml')
    
    content=soup.find('ul',class_='news')
    
    for b in content.find_all('li',class_='news-item'):
        headline=b.find('h2')
        print(headline.text)

In [6]:
anime=Anime()
anime

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\asus\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Anime_Name,Number of Videos Aired,Link,Reviews,Polarity Score,Sentiment
0,RADIANT,31 Videos,https://www.crunchyroll.com/radiant,Read the Manga if you want the FULL experience...,0.197508,Mixed Reviews
1,BORUTO: NARUTO NEXT GENERATIONS,134 Videos,https://www.crunchyroll.com/boruto-naruto-next...,If Boruto follows Naruto in terms of episodes ...,-0.0282656,Negative Reviews
2,Naruto Shippuden,500 Videos,https://www.crunchyroll.com/naruto-shippuden,Sorry Naruto you deserved better An amazing ri...,0.265613,Mixed Reviews
3,JoJo's Bizarre Adventure,77 Videos,https://www.crunchyroll.com/jojos-bizarre-adve...,SONO CHI NO SADAME!! A faithful adaptation of ...,0.401794,Positive Reviews
4,DARLING in the FRANXX,120 Videos,https://www.crunchyroll.com/darling-in-the-franxx,A Unique and Telling Twist on Our Own Modern R...,0.345814,Positive Reviews
5,Monster Strike,155 Videos,https://www.crunchyroll.com/monster-strike,Unfulfilled plot premise Helps me fall asleep ...,-0.0805769,Negative Reviews
6,The Ancient Magus' Bride,108 Videos,https://www.crunchyroll.com/the-ancient-magus-...,i REALLY like this but a lot of you have the w...,0.476562,Positive Reviews
7,ReLIFE,35 Videos,https://www.crunchyroll.com/relife,Ever feel like repeating a year in high school...,0.410859,Positive Reviews
8,Eyeshield 21,145 Videos,https://www.crunchyroll.com/eyeshield-21,Definetly my favorite anime Eyeshieldo kun ich...,0.513054,Very Positive Reviews
9,Demon King Daimao,12 Videos,https://www.crunchyroll.com/demon-king-daimao,Very Interesting so far! An interesting idea g...,0.372326,Positive Reviews


In [13]:
manga=Manga()
manga

C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Manga Name,Link,Overview
0,Domestic Girlfriend,https://www.crunchyroll.com/comics/manga/domes...,High schooler Natsuo is hopelessly in love wit...
1,The Seven Deadly Sins,https://www.crunchyroll.com/comics/manga/the-s...,When they were accused of trying to overthrow ...
2,Attack on Titan,https://www.crunchyroll.com/comics/manga/attac...,"A century ago, the grotesque giants known as T..."
3,EDENS ZERO,https://www.crunchyroll.com/comics/manga/edens...,Eleven months after the grand finale of Fairy ...
4,One Room of Happiness,https://www.crunchyroll.com/comics/manga/one-r...,"That day, she was abducted. But to her, that a..."
5,Inside Mari,https://www.crunchyroll.com/comics/manga/insid...,"A young man is a shut-in, with nothing to do b..."
6,Tales of Wedding Rings,https://www.crunchyroll.com/comics/manga/tales...,Kekkon Yubiwa Monogatari - A high school girl ...
7,Knight's & Magic,https://www.crunchyroll.com/comics/manga/knigh...,A genius programmer and hardcore robot otaku i...
8,Fairy Tail,https://www.crunchyroll.com/comics/manga/fairy...,Cute girl wizard Lucy wants to join the Fairy ...
9,The Rising of the Shield Hero: The Manga Compa...,https://www.crunchyroll.com/comics/manga/the-r...,"Naofumi Iwatani, an uncharismatic otaku who sp..."


In [19]:
similarity_distance=recommendation(manga)
results=get_recommendation('Inside Mari',similarity_distance)
results=results.reset_index()
results.drop('index',axis=1,inplace=True)
results=results.iloc[1:10,]
results

,Manga Name,Link,Overview,ID
1,Silver Nina,https://www.crunchyroll.com/comics/manga/silve...,Shimazaki Shutaro goes back home to the countr...,1032
2,Hoshi no Samidare: The Lucifer and Biscuit Hammer,https://www.crunchyroll.com/comics/manga/hoshi...,Asamiya Yuuhi was an ordinary college student....,1047
3,One Room of Happiness,https://www.crunchyroll.com/comics/manga/one-r...,"That day, she was abducted. But to her, that a...",1004
4,Girl May Kill,https://www.crunchyroll.com/comics/manga/girl-...,A port town where the Mafia actively operates ...,1040
5,Domestic Girlfriend,https://www.crunchyroll.com/comics/manga/domes...,High schooler Natsuo is hopelessly in love wit...,1000
6,Spirit Circle,https://www.crunchyroll.com/comics/manga/spiri...,Fuuta Okeya is a normal middle school 2nd grad...,1043
7,The Daily Life of Crunchyroll-Hime,https://www.crunchyroll.com/comics/manga/the-d...,A bi-monthly Japanese style yonkoma comedy man...,1018
8,To Your Eternity,https://www.crunchyroll.com/comics/manga/to-yo...,A new manga from the creator of the acclaimed ...,1028
9,Crossing Time,https://www.crunchyroll.com/comics/manga/cross...,This is a collection of slice-of-life omnibus ...,1023


In [22]:
url_news='https://www.crunchyroll.com/news'
latest_news(url_news)


Mai Fuchigami Sings TV Anime Sorcerous Stabber Orphen ED Song in Empty Schoolhouse


The Akai Family Unites in Detective Conan: The Scarlet Bullet Film Teaser Trailer 


Gainax President Maki Tomohiro Arrested on ‘Semi-Forced’ Indecent Assault Charges  


Yuki Kaji is The Most Searched Voice Actor on Yahoo Japan in 2019 


Voice Actor Daisuke Ono Releases His 12th Single "Deep & Holic" MV


Crunchyroll Editorial's Top 100 Anime of the Decade (By The Numbers!)


Trails of Cold Steel III Is a Thrilling Build to a Climactic Ending


PlayStation JP Takes Down Music Video Amidst Art Theft Accusations


Why Asuka is One of the Best Anime Characters of All Time


The History of Hentai


Weathering with You, PROMARE, and More Nominated for Annie Awards


Hypnosis Mic Brings the Battle to TV in 2020 Anime Series


The Case Files of Jeweler Richard Displays a Sparkling New PV


Crunchyroll Becomes Majority Owner of VIZ Media Europe Group


Only the Strong Survive in Murenase! Seton Gakuen TV An

In [20]:
anime.to_csv('Anime_List.csv')
manga.to_csv('Manga_List.csv')